In [2]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from infodynamics import WindowedRollingDistance
from infodynamics.util import calc_vector_histogram

from src.dataset.util import read_jsonl, write_jsonl
from src.model.document_order import generate_permutations_by_label


DATA_PROCESSED = '../data/processed/'
DATA_INTERIM = '../data/interim/'

In [3]:
### load data
ds = read_jsonl(os.path.join(DATA_PROCESSED, 'publications_merged_concat.ndjson'))
ds = pd.DataFrame(ds)
ds[['id', 'section']] = ds['id'].str.split('_', expand=True) 

### meta
meta = pd.read_csv(os.path.join(DATA_INTERIM, 'meta_publications_merged.csv'))
meta['year'] = [funk.year for funk in pd.to_datetime(meta['reconstructed_date'], format='mixed')]

### merge
df = ds.merge(meta[['id', 'year', 'source']], on='id', how='left')
df = df.sort_values('year')

In [16]:
subset = df.query('warning == "ok :)"').query('section == "b"')
X = subset['embeddings'].tolist()
X = np.array(X)

In [4]:
wrd = WindowedRollingDistance(window_size=5, measure='jensenshannon')
sig = wrd.fit_transform(X)

subset['N_hat'] = sig['N_hat']

/Users/au582299/Repositories/friths/.venv/lib/python3.11/site-packages/infodynamics/metrics.py:19: RuntimeWarning: invalid value encountered in log2
  result = np.where(mask, p * np.log2(p / q), 0)


In [10]:
perm = generate_permutations_by_label(
    labels=subset['year'].to_list(),
    datapoints=subset['embeddings'].to_list(),
)


In [6]:
# list(perm.keys())
perm.keys()

dict_keys([1969, 1970, 1971, 1972, 1973, 1974, 1975, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2018, 2019, 2020])

In [7]:
a = list(perm[1970])

[
    # n permutations
    len(a),
    # n documents
    len(a[0]),
    # n dimensions
    len(a[0][0])
]

[6, 3, 1536]

In [11]:
# years_possible = list(perm.keys())
# all_signal_permutations = []

# for year_of_focus in tqdm(years_possible):

#     for permutation in list(perm[year_of_focus]):
#         single_signal = []

#         for year in years_possible:
#             if year_of_focus == year:
#                 single_dp = permutation
#                 single_signal.append(single_dp)
#             else:
#                 single_dp = list(perm[year])[0]
#                 single_signal.append(single_dp)
        
#             all_signal_permutations.append(single_signal)

In [8]:
year_of_focus = 1969
idx = subset.query('year == @year_of_focus').index

In [17]:
subset_2 = subset.copy(deep=True)
subset_2.loc[idx, 'embeddings'] = [['a'], ['b']]

subset_2['embeddings'].tolist()